ref: https://langfuse.com/guides/cookbook/example_pydantic_ai_mcp_agent_evaluation

- trajactory(trace): Agent의 전체적인 시퀀스
- Agent 개발에서의 문제점은
    1. understanding: 정확히 어떤 툴을 호출하고 어떤 추론을 하는 trajactory에 대해 사용자가 모르는 경우가 많다.
    2. specification: 프롬프트나 few-shot에서 좋은 action에 대한 내용이 분명하게 말하지 않아서 agent가 예측하기 어렵게 된다.
    3. generalization: 일부의 평가 데이터에서는 잘 동작하더라도 실제 환경에서 실패하는 경우가 많다.
- 평가 3단계
    - Phase 1: Early Developoment (Manual Tracing)
        - 개발 단계에서 trajactory를 계속 확인하면서 agent 추론 과정을 보며 인사이트 얻는다.
    - Phase 2: First Users (Online Evaluation)
        - 사용자 반응 데이터를 얻고, 문제가 있을 것 같은 trajactory를 리뷰한다.
    - Phase 3: Scaling (Offline Evaluation)
        - 서비스 스케일이 커지면서 모든 trajactory를 리뷰하기 어려워지는데, 표준 평가 데이터셋을 구축해두고, trajactory review를 자동화시킨다.
- Langfuse는 phase 3의 offline evaluation에 초점이 맞춰져있다. 평가 자동화 전략으로는
    - Final Response: 사용자 input과 Agent Output만 보고 평가하는 전략으로 간단하면서 유연하지만, agent가 왜 failed했는지 알 수 없다.
    - Tranjactory: 내부 동작 시퀀스를 관찰하며 왜 실패했는지 찾을 수 있도록 돕는다.
    - Single Step: reasoning 단계까지 모니터링하며 decision making 과정까지 살펴보고 왜 이러한 action이 나왔는지 살펴본다.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGFUSE_PUBLIC_KEY = os.getenv("LANGFUSE_PUBLIC_KEY")
LANGFUSE_SECRET_KEY = os.getenv("LANGFUSE_SECRET_KEY")
LANGFUSE_HOST = os.getenv("LANGFUSE_HOST")

In [2]:
from langfuse import get_client
from pydantic_ai.agent import Agent
 
langfuse = get_client()
assert langfuse.auth_check(), "Langfuse auth failed - check your keys"
 
Agent.instrument_all()
print("✅ Pydantic AI instrumentation enabled")

✅ Pydantic AI instrumentation enabled


In [ ]:
# create agent

from typing import Any
from pydantic_ai import Agent, RunContext
from pydantic_ai.mcp import MCPServerStreamableHTTP, CallToolFunc, ToolResult
 
LANGFUSE_MCP_URL = "https://langfuse.com/api/mcp"
 
async def run_agent(item, system_prompt="You are an expert on Langfuse. ", model="openai:gpt-4o-mini"):
    langfuse.update_current_trace(input=item.input)
 
    tool_call_history = []
 
    async def process_tool_call(
        ctx: RunContext[Any],
        call_tool: CallToolFunc,
        tool_name: str,
        args: dict[str, Any],
    ) -> ToolResult:
        tool_call_history.append({"tool_name": tool_name, "args": args})
        return await call_tool(tool_name, args)
    
    langfuse_docs_server = MCPServerStreamableHTTP(
        url=LANGFUSE_MCP_URL,
        process_tool_call=process_tool_call,
    )
 
    agent = Agent(
        model=model,
        system_prompt=system_prompt,
        toolsets=[langfuse_docs_server],
    )
 
    async with agent:
        result = await agent.run(item.input["question"])
        
        langfuse.update_current_trace(
            output=result.output,
            metadata={"tool_call_history": tool_call_history},
        )
 
        return result.output, tool_call_history

In [ ]:
# create evaluation dataset

test_cases = [
    {
        "input": {"question": "What is Langfuse?"},
        "expected_output": {
            "response_facts": [
                "Open Source LLM Engineering Platform",
                "Product modules: Tracing, Evaluation and Prompt Management"
            ],
            "trajectory": ["getLangfuseOverview"],
        }
    },
    {
        "input": {"question": "How to trace a python application with Langfuse?"},
        "expected_output": {
            "response_facts": [
                "Python SDK, you can use the observe() decorator",
                "Lots of integrations, LangChain, LlamaIndex, Pydantic AI, and many more."
            ],
            "trajectory": ["getLangfuseOverview", "searchLangfuseDocs"],
            "search_term": "Python Tracing"
        }
    },
    {
        "input": {"question": "How to connect to the Langfuse Docs MCP server?"},
        "expected_output": {
            "response_facts": [
                "Connect via the MCP server endpoint: https://langfuse.com/api/mcp",
                "Transport protocol: `streamableHttp`"
            ],
            "trajectory": ["getLangfuseOverview"]
        }
    },
    {
        "input": {"question": "How long are traces retained in langfuse?"},
        "expected_output": {
            "response_facts": [
                "By default, traces are retained indefinitely",
                "You can set custom data retention policy in the project settings"
            ],
            "trajectory": ["getLangfuseOverview", "searchLangfuseDocs"],
            "search_term": "Data retention"
        }
    }
]
 
DATASET_NAME = "pydantic-ai-mcp-agent-evaluation"
 
dataset = langfuse.create_dataset(name=DATASET_NAME)
for case in test_cases:
    langfuse.create_dataset_item(
        dataset_name=DATASET_NAME,
        input=case["input"],
        expected_output=case["expected_output"]
    )

In [9]:
# set evaluator in langfuse UI ('LLM-as-a-judge' > 'set up evaluator')

#Final Response Evaluation (Black Box)
"""
You are a teacher grading a student based on the factual correctness of their statements.
 
### Examples
 
#### Example 1:
- Response: "The sun is shining brightly."
- Facts to verify: ["The sun is up.", "It is a beautiful day."]
- Reasoning: The response includes both facts.
- Score: 1
 
#### Example 2:
- Response: "When I was in the kitchen, the dog was there"
- Facts to verify: ["The cat is on the table.", "The dog is in the kitchen."]
- Reasoning: The response mentions the dog but not the cat.
- Score: 0
 
### New Student Response
 
- Response: {{response}}
- Facts to verify: {{facts_to_verify}}
"""

## Tranjactory Evaluation (Glass Box)
"""
You are comparing two lists of strings. Check whether the lists contain exactly the same items. Order does not matter.
 
## Examples
 
Expected: ["searchWeb", "visitWebsite"]
Output: ["searchWeb"]
Reasoning: Output missing "visitWebsite".
Score: 0
 
Expected: ["drawImage", "visitWebsite", "speak"]
Output: ["visitWebsite", "speak", "drawImage"]
Reasoning: Output matches expected items.
Score: 1
 
Expected: ["getNews"]
Output: ["getNews", "watchTv"]
Reasoning: Output contains unexpected "watchTv".
Score: 0
 
## This Exercise
 
Expected: {{expected}}
Output: {{output}}
"""

## Search Quality Evaluation
"""
You are grading whether a student searched for the right information. The search term should correspond vaguely with the expected term.
 
### Examples
 
Response: "How can I contact support?"
Expected search topics: Support
Reasoning: Response searches for support.
Score: 1
 
Response: "Deployment"
Expected search topics: Tracing
Reasoning: Response doesn't match expected topic.
Score: 0
 
Response: (empty)
Expected search topics: (empty)
Reasoning: No search expected, no search done.
Score: 1
 
### New Student Response
 
Response: {{search}}
Expected search topics: {{expected_search_topic}}
"""

'\nYou are grading whether a student searched for the right information. The search term should correspond vaguely with the expected term.\n\n### Examples\n\nResponse: "How can I contact support?"\nExpected search topics: Support\nReasoning: Response searches for support.\nScore: 1\n\nResponse: "Deployment"\nExpected search topics: Tracing\nReasoning: Response doesn\'t match expected topic.\nScore: 0\n\nResponse: (empty)\nExpected search topics: (empty)\nReasoning: No search expected, no search done.\nScore: 1\n\n### New Student Response\n\nResponse: {{search}}\nExpected search topics: {{expected_search_topic}}\n'

In [10]:
# run experiments

dataset = langfuse.get_dataset(DATASET_NAME)
 
result = dataset.run_experiment(
    name="Production Model Test3",
    description="Monthly evaluation of our production model",
    task=run_agent
)
 
print(result.format())

Individual Results: Hidden (2 items)\n💡 Set include_item_results=True to view them\n\n──────────────────────────────────────────────────\n🧪 Experiment: Production Model Test3
📋 Run name: Production Model Test3 - 2025-12-04T06:13:22.101896Z - Monthly evaluation of our production model\n2 items\n🔗 Dataset Run:\n   https://cloud.langfuse.com/project/cmiqs4ss0000xad07duzdlnpf/datasets/cmiqsg110001iad07w45c8vw9/runs/3a65e040-6636-4e53-a5ba-db17eeba2726


In [8]:
# compare multiple configurations

from functools import partial
 
system_prompts = {
    "simple": (
        "You are an expert on Langfuse. "
        "Answer user questions accurately and concisely using the available MCP tools. "
        "Cite sources when appropriate."
    ),
    "nudge_search": (
        "You are an expert on Langfuse. "
        "Answer user questions accurately and concisely using the available MCP tools. "
        "Always cite sources when appropriate. "
        "When unsure, use getLangfuseOverview then search the docs. You can use these tools multiple times."
    )
}
 
models = ["openai:gpt-5-mini", "openai:gpt-5-nano"]
 
dataset = langfuse.get_dataset(DATASET_NAME)
 
for prompt_name, prompt_content in system_prompts.items():
    for test_model in models:
        task = partial(
            run_agent,
            system_prompt=prompt_content,
            model=test_model,
        )
 
        result = dataset.run_experiment(
            name=f"Test: {prompt_name} {test_model}",
            description="Comparing prompts and models",
            task=task
        )
 
        print(result.format())

Individual Results: Hidden (1 items)\n💡 Set include_item_results=True to view them\n\n──────────────────────────────────────────────────\n🧪 Experiment: Test: simple openai:gpt-5-mini
📋 Run name: Test: simple openai:gpt-5-mini - 2025-12-04T03:28:25.883522Z - Comparing prompts and models\n1 items\n🔗 Dataset Run:\n   https://cloud.langfuse.com/project/cmiqs4ss0000xad07duzdlnpf/datasets/cmiqsg110001iad07w45c8vw9/runs/f3314f0d-32c2-4e5a-96ae-8f8fe23f4d07
Individual Results: Hidden (2 items)\n💡 Set include_item_results=True to view them\n\n──────────────────────────────────────────────────\n🧪 Experiment: Test: simple openai:gpt-5-nano
📋 Run name: Test: simple openai:gpt-5-nano - 2025-12-04T03:33:31.071440Z - Comparing prompts and models\n2 items\n🔗 Dataset Run:\n   https://cloud.langfuse.com/project/cmiqs4ss0000xad07duzdlnpf/datasets/cmiqsg110001iad07w45c8vw9/runs/6e399d08-cee9-49f3-83f9-ab5e6c15a7cc
Individual Results: Hidden (2 items)\n💡 Set include_item_results=True to view them\n\n──────